In [3]:
import tensorflow as tf

MODEL_PATH = "backend/models/vgg16_model2.h5"
model = tf.keras.models.load_model(MODEL_PATH)

# 1️⃣ Extraer el submodelo VGG16
vgg = model.get_layer("vgg16")
print("Submodelo VGG16 cargado")

# 2️⃣ Listar capas convolucionales DENTRO del submodelo
conv_layers = [
    layer.name
    for layer in vgg.layers
    if isinstance(layer, tf.keras.layers.Conv2D)
]

print("\nCapas Conv2D en VGG16:")
for name in conv_layers:
    print(" -", name)

# 3️⃣ Seleccionar última capa convolucional real
last_conv_layer_name = conv_layers[-1]
print("\nÚltima capa Conv:", last_conv_layer_name)

# 4️⃣ Construir modelo Grad-CAM CORRECTO
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[
        vgg.get_layer(last_conv_layer_name).output,
        model.output
    ]
)

# 5️⃣ Test de gradientes
import numpy as np
dummy = tf.random.uniform((1, 224, 224, 3))

with tf.GradientTape() as tape:
    conv_outputs, preds = grad_model(dummy)
    class_idx = tf.argmax(preds[0])
    loss = preds[:, class_idx]

grads = tape.gradient(loss, conv_outputs)

print("\nGradients shape:", grads.shape if grads is not None else None)

if grads is not None:
    print("\n✅ GRAD-CAM ES TOTALMENTE VIABLE EN ESTE MODELO")
else:
    print("\n❌ Gradientes no disponibles")


Submodelo VGG16 cargado

Capas Conv2D en VGG16:
 - block1_conv1
 - block1_conv2
 - block2_conv1
 - block2_conv2
 - block3_conv1
 - block3_conv2
 - block3_conv3
 - block4_conv1
 - block4_conv2
 - block4_conv3
 - block5_conv1
 - block5_conv2
 - block5_conv3

Última capa Conv: block5_conv3


KeyError: "Exception encountered when calling Functional.call().\n\n\x1b[1m130290991036752\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(1, 224, 224, 3), dtype=float32)\n  • training=None\n  • mask=None\n  • kwargs=<class 'inspect._empty'>"